<a href="https://colab.research.google.com/github/AncestorComposition/public/blob/main/Ancestor_Composition_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Estimate the ancestor composition of your genome using DNA match segment data and information about common ancestors.*
### Step 1: Upload CSV files
*   Click on the folder icon to the left to expand the "Files" pane.
*   Click the "Upload to session storage" button in the upper left to upload CSV files for match shared segments (from MyHeritage, FamilyTree DNA, and/or 23andMe) and a CSV file with common ancestor information for some of your matches. [Detailed directions](#directions) are provided below.

In [1]:
!pip install distinctipy -q
#@title Step 2: Calculate your Ancestor Composition { display-mode: "form" }
from bokeh.models import AnnularWedge, HoverTool, Range1d, Panel, Tabs
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models.widgets.tables import ColumnDataSource
import pandas as pd
from distinctipy import distinctipy, get_hex
from collections import defaultdict
import requests
import math
import os
Gender = "Male"  # @param ["Male", "Female"]
OptionalAPIKey = "" # @param {type:"string"}

output_notebook()

sidemap = {}

groups_rep = list(range(4, 64))

allchromes = list(range(1, 23))
allchromes.append('X')

def gen(anc):
    '''Generation of ancestor '''
    return math.floor(math.log(anc, 2))


def ahnentafel_descendants(a_val, a_min):
    ''' Ahnentafel numbers of descendants '''
    if not isinstance(a_val, list):
        a_val = [a_val]
    if not a_val[-1] % 2 == 0:
        a_val.append(int((a_val[-1] - 1) / 2))
    else:
        a_val.append(int(a_val[-1] / 2))
    if a_val[-1] <= a_min:
        return a_val
    return ahnentafel_descendants(a_val, a_min)


def side(a_val):
  ''' Determine whether ancestor is on paternal or maternal side '''
  if a_val in sidemap:
      return sidemap[a_val]
  if 2 in ahnentafel_descendants(a_val, 0):
      sidemap[a_val] = 'Paternal'
      return 'Paternal'
  sidemap[a_val] = 'Maternal'
  return 'Maternal'


def csv_reader():
  ''' Parse CSV files '''
  files = os.listdir()
  segsmh = None
  segsftdna = None
  segs23 = None
  mf = None
  allnames = set()
  for f in files:
      if '.csv' in f:
          try:
              df = pd.read_csv(f)
          except Exception:
              pass
          if 'Match name' in df and 'SNPs' in df:
              segsmh = df
              allnames = allnames.union(set(list(segsmh['Match name'])))
          elif 'Match Name' in df and 'Matching SNPs' in df:
              segsftdna = df
              allnames = allnames.union(set(list(segsftdna['Match Name'])))
          elif 'Display Name' in df and 'Genetic Distance' in df:
              segs23 = df
              allnames = allnames.union(set(list(segs23['Display Name'])))
          elif 'Male Ancestor #' in df and len(df) > 4:
              mf = df
  if not allnames:
    raise Exception("Did not find any valid match segment CSV files.")
  if not isinstance(mf, pd.DataFrame):
    raise Exception("Did not find a valid common ancestor file.")
  #Obscure the match names
  obscure_dict = dict(zip(allnames, range(len(allnames))))

  # FTDNA matches and segs
  matches = {}
  if isinstance(segsftdna, pd.DataFrame):
    for (index, row) in segsftdna.iterrows():
        nmind = obscure_dict[row['Match Name']]
        if not nmind in matches:
            matches[nmind] = {}
            matches[nmind]['cm'] = 0
            matches[nmind]['Segs'] = defaultdict(list)
        matches[nmind]['cm'] = matches[nmind]['cm'] \
            + row['Centimorgans']
        matches[nmind]['Segs'][str(row['Chromosome'])].append((row['Start Location'], row['End Location'],
                                                              row['Centimorgans'], row['Matching SNPs']))
  # MH matches and segs
  if isinstance(segsmh, pd.DataFrame):
    for (index, row) in segsmh.iterrows():
        nmind = obscure_dict[row['Match name']]
        if not nmind in matches:
            matches[nmind] = {}
            matches[nmind]['cm'] = 0
            matches[nmind]['Segs'] = defaultdict(list)
        matches[nmind]['cm'] = matches[nmind]['cm'] \
            + row['Centimorgans']
        matches[nmind]['Segs'][str(row['Chromosome'
                                       ])].append((row['Start Location'], row['End Location'],
                                                   row['Centimorgans'], row['SNPs']))
  # 23andMe
  if isinstance(segs23, pd.DataFrame):
    for (index, row) in segs23.iterrows():
        nmind = obscure_dict[row['Display Name']]
        if not nmind in matches:
            matches[nmind] = {}
            matches[nmind]['cm'] = 0
            matches[nmind]['Segs'] = defaultdict(list)
        if isinstance(row['Chromosome Number'], str):
            matches[nmind]['cm'] = matches[nmind]['cm'] + \
                row['Genetic Distance']
            matches[nmind]['Segs'][row['Chromosome Number'
                                       ]].append((row['Chromosome Start Point'],
                                                  row['Chromosome End Point'],
                                                  row['Genetic Distance'], row['# SNPs']))

  # Common ancestor data
  groupdict = dict.fromkeys(range(4, 64), 'Unnamed Ancestor')
  commonancestors = dict()
  for (index, row) in mf.iterrows():
      groupdict[row['Male Ancestor #']] = row['Male Ancestor Name']
      groupdict[row['Female Ancestor #']] = row['Female Ancestor Name']
      if row['Match Name'] in obscure_dict:
        nmind = obscure_dict[row['Match Name']]
        if nmind in matches:
          if row['Male Ancestor #'] > 0 and row['Female Ancestor #'] > 0:
            commonancestors[nmind] = int(row['Male Ancestor #']/2)
          elif row['Male Ancestor #'] > 0:
            commonancestors[nmind] = int(row['Male Ancestor #'])
          elif row['Female Ancestor #'] > 0:
            commonancestors[nmind] = int(row['Female Ancestor #'])

  return matches, groupdict, commonancestors


matches, groupdict, commonancestors = csv_reader()


def call_ancestor_composition_solver(matches, n, Gender, commonancestors):
  # Call the Ancestor Composition Solver API
  token = OptionalAPIKey
  sort_matches = sorted(matches.items(), key=lambda x: x[1]['cm'], reverse=True)
  first_n = dict(list(sort_matches)[:n])
  json_input = dict()
  url = "https://5if5u6ou58.execute-api.us-east-1.amazonaws.com/default/AncestorComposition"
  json_input["Matches"] = first_n
  json_input["Gender"] = Gender
  json_input["Token"] = token
  json_input["commonancestors"] = commonancestors
  r = requests.post(url, json=json_input)
  json_output = r.json()
  ancestor_content = json_output["ancestor_content"]
  ancestor_segments = json_output["ancestor_segments"]
  message = json_output["message"]
  return ancestor_content, ancestor_segments, message


if len(OptionalAPIKey) < 16:
  n = 1000
else:
  n = 30000
ancestor_content, ancestor_segments, message = call_ancestor_composition_solver(
    matches, n, Gender, commonancestors)


def plot_results(ancestor_content, ancestor_segments):
  '''Display the results'''
  if Gender == "Male":
    totbp = 6.27e9
  else:
    totbp = 6.37e9
  tups = []
  tups.append((1, 'You', 100))
  if Gender == "Male":
    tups.append((2, 'Father', 49.23))
    tups.append((3, 'Mother', 50.76))
  elif Gender == "Female":
    tups.append((2, 'Father', 50.0))
    tups.append((3, 'Mother', 50.0))

  for g in ancestor_content:
      gn = int(g)
      if gn % 2 == 0:
          content = 100 * (62460029 + ancestor_content[g]) / totbp
      else:
          content = 100 * ancestor_content[g] / totbp
      if gn in groupdict:
          tups.append((gn, groupdict[gn], content))
      else:
          tups.append((gn, ' ', content))

  colors = distinctipy.get_colors(len(groups_rep), rng=1)
  colorhex = {}
  for (ind, g) in enumerate(groups_rep):
      colorhex[g] = get_hex(colors[ind])

  # Plot chromosomes
  p = figure(width=900, height=2000, tools='save')
  p.toolbar.logo = None

  for g, gcolor in colorhex.items():
      ws = list()
      ys = list()
      xs = list()
      gens = gen(g) - 1
      if side(g) == 'Paternal':
          MULT = 1
      else:
          MULT = -1
      for c in allchromes + ['Y']:
          if str(c) in ancestor_segments[str(g)]:
              for seg in ancestor_segments[str(g)][str(c)]:
                  if c in range(1, 23):
                      ws.append((seg[1] - seg[0]) / 1e6)
                      ys.append(c + MULT * gens / 12)
                      xs.append((seg[0] + seg[1]) / 2e6)
                  elif c == 'X':
                      ws.append((seg[1] - seg[0]) / 1e6)
                      ys.append(MULT * gens / 12)
                      xs.append((seg[0] + seg[1]) / 2e6)
                  elif c == 'Y':
                      MULT = 1
                      ws.append((seg[1] - seg[0]) / 1e6)
                      ys.append(MULT * gens / 12)
                      xs.append((seg[0] + seg[1]) / 2e6)
      p.rect(
          x=xs,
          y=ys,
          height=0.08,
          width=ws,
          color=gcolor,
          name=groupdict[g],
          legend_label=groupdict[g],
      )

  hover_tool = HoverTool(tooltips='$name')
  hover_tool.point_policy = 'follow_mouse'
  p.tools.append(hover_tool)
  p.yaxis.axis_label = 'Chromosome #'
  p.xaxis.axis_label = 'Mbp'
  p.x_range = Range1d(0, 270)
  p.legend.location = 'top_right'
  p.legend.click_policy = 'hide'
  hover_tool.names = list(groupdict.values())
  p.y_range = Range1d(-0.5, 22.5)

  # Plot  chromosome boundaries
  ws = list()
  ys = list()
  xs = list()
  cw = {
      1: 255,
      2: 246,
      3: 201.1,
      4: 193.5,
      5: 182.0,
      6: 172.1,
      7: 160.6,
      8: 146.3,
      9: 150.6,
      10: 138,
      11: 136.1,
      12: 138,
      13: 120,
      14: 110,
      15: 105,
      16: 96.3,
      17: 84.3,
      18: 80.5,
      19: 61.7,
      20: 66.2,
      21: 48,
      22: 51.3,
  }
  for c in range(1, 23):
      ws.append(cw[c])
      ys.append(c + 0.20)
      xs.append(cw[c] / 2)

  # X or Y chromosome, depending on gender
  if Gender == "Male":
    ws.append(58)
    ys.append(0.20)
    xs.append(58 / 2)
    p.rect(
        x=xs,
        y=ys,
        height=0.40,
        width=ws,
        line_color='white',
        color='blue',
        fill_alpha=0.05,
        name='father',
    )
  else:
    ws.append(155)
    ys.append(0.20)
    xs.append(155 / 2)
    p.rect(
        x=xs,
        y=ys,
        height=0.40,
        width=ws,
        line_color='white',
        color='blue',
        fill_alpha=0.05,
        name='father',
    )
  # Plot maternal chromosome boundaries
  ws = list()
  ys = list()
  xs = list()

  for c in range(1, 23):
      ws.append(cw[c])
      ys.append(c - 0.20)
      xs.append(cw[c] / 2)

  # Maternal X chromosome

  ws.append(155)
  ys.append(-0.20)
  xs.append(155 / 2)
  p.rect(
      x=xs,
      y=ys,
      height=0.40,
      width=ws,
      line_color='white',
      color='red',
      fill_alpha=0.05,
      name='mother',
  )

  p.xgrid.grid_line_color = None
  p.yaxis.ticker = list(range(23))
  p.legend.title = 'Ancestors'
  if Gender == "Male":
    p.yaxis.major_label_overrides = {0: 'Y/X'}
  else:
    p.yaxis.major_label_overrides = {0: 'X'}
  tabs = []
  tab1 = Panel(child=p, title='Chromosomes')
  tabs.append(tab1)

 #Annular wedge chart showing inheritance by generation
  p2 = figure(
      height=900,
      width=900,
      toolbar_location=None,
      tools='hover',
      tooltips='@name: @pct{1.1}%',
      x_range=(-1.5, 1.5),
      y_range=(-1.5, 1.5),
  )
  sadict = {}
  eadict = {}

  eadict[2] = math.pi
  if Gender == "Male":
    sadict[2] = math.pi - 49.23 * math.pi / 100
    eadict[3] = 50.76 * math.pi / 100
    tups.append((2, 'Father', 49.23))
    tups.append((3, 'Mother', 50.76))
  else:
    sadict[2] = math.pi - 50.0 * math.pi / 100
    eadict[3] = 50.00 * math.pi / 100
    tups.append((2, 'Father', 50))
    tups.append((3, 'Mother', 50))

  sadict[3] = 0
  # For each generation
  for GENID in range(2, 6):
      angle = []
      color = []
      name = []
      startangle = []
      endangle = []
      pct = []
      start = -math.pi
      for (ind, t) in enumerate([t for t in tups if gen(t[0]) == GENID]):
          if True:
              if t[0] % 2 == 0:  # male
                  end = eadict[int(t[0] / 2)]
                  endangle.append(end)
                  start = end - t[2] * math.pi / 100
                  startangle.append(start)
              else:
                # female
                  start = sadict[int((t[0] - 1) / 2)]
                  startangle.append(start)
                  end = start + t[2] * math.pi / 100
                  endangle.append(end)
              angle.append(t[2] * math.pi / 100)
              color.append(colorhex[t[0]])
              pct.append(t[2])
              name.append(t[1])
              sadict[t[0]] = start
              eadict[t[0]] = end

      md = {
          'startangle': startangle,
          'color': color,
          'name': name,
          'endangle': endangle,
          'pct': pct,
      }
      df = pd.DataFrame(md)
      source = ColumnDataSource(df)

      glyph = AnnularWedge(
          x=0,
          y=0,
          inner_radius=0.1 + 0.20 * GENID,
          outer_radius=0.3 + 0.20 * GENID,
          start_angle='startangle',
          end_angle='endangle',
          line_color='white',
          fill_color='color',
      )
      p2.add_glyph(source, glyph)

  angle = []
  color = []
  name = []
  startangle = []
  endangle = []
  pct = []

  startangle.append(0)
  name.append('Mother')
  if Gender == "Male":
    pct.append(50.76)
    endangle.append(50.76 * math.pi / 100)
  else:
    pct.append(50.00)
    endangle.append(50.00 * math.pi / 100)
  color.append('#FFC0CB')

  endangle.append(math.pi)
  name.append('Father')
  if Gender == "Male":
    pct.append(49.23)
    startangle.append(50.76 * math.pi / 100)
  else:
    pct.append(50.00)
    startangle.append(50.00 * math.pi / 100)
  color.append('#ADD8E6')
  md = {
      'startangle': startangle,
      'color': color,
      'name': name,
      'endangle': endangle,
      'pct': pct,
  }
  df = pd.DataFrame(md)
  source = ColumnDataSource(df)

  GENID = 1
  glyph = AnnularWedge(
      x=0,
      y=0,
      inner_radius=0.1 + 0.20 * GENID,
      outer_radius=0.3 + 0.20 * GENID,
      start_angle='startangle',
      end_angle='endangle',
      line_color='white',
      fill_color='color',
  )
  p2.add_glyph(source, glyph)

  p2.axis.axis_label = None
  p2.axis.visible = False
  p2.grid.grid_line_color = None
  tab = Panel(child=p2, title='Inheritance')
  tabs.append(tab)

  show(Tabs(tabs=tabs))

plot_results(ancestor_content, ancestor_segments)

print(message)

def diagnostic_messages(ancestor_content, ancestor_segments, groupdict):
  if Gender == "Male":
    totbp = 6.27e9
  else:
    totbp = 6.37e9
  print("*** Coverage ***")
  gp = 0
  for c in range(4, 8):
    gp = gp+ancestor_content[str(c)]
  print('Coverage at grandparent generation is ' +
        "{:.1f}".format(100*gp/totbp) + "%")
  unallocgp = 100-100*gp/totbp

  ggp = 0
  for c in range(8, 16):
    ggp = ggp+ancestor_content[str(c)]
  print('Coverage at great-grandparent generation is ' +
        "{:.1f}".format(100*ggp/totbp) + "%")
  unallocggp = 100-100*ggp/totbp

  gggp = 0
  for c in range(16, 32):
    gggp = gggp+ancestor_content[str(c)]
  print('Coverage at 2nd great-grandparent generation is ' +
        "{:.1f}".format(100*gggp/totbp) + "%")

  ggggp = 0
  for c in range(32, 64):
    ggggp = ggggp+ancestor_content[str(c)]
  print('Coverage at 3rd great-grandparent generation is ' +
        "{:.1f}".format(100*ggggp/totbp) + "%")

  if unallocgp < 15:
    pcov = 0
    for c in range(1, 23):
      pcov = pcov+min([len(ancestor_segments['4'][str(c)]),
                      len(ancestor_segments['5'][str(c)])])

    mcov = 0
    for c in range(1, 23):
      mcov = mcov+min([len(ancestor_segments['6'][str(c)]),
                      len(ancestor_segments['7'][str(c)])])
    mcov = mcov+min([len(ancestor_segments['6']['X']),
                    len(ancestor_segments['7']['X'])])

    print("*** Crossovers ***")
    print("Your genome has " + str(pcov) + " paternal crossovers and " +
          str(mcov) + " maternal crossovers.")
    print("The average number of paternal crossovers is 26.4 and the average number of maternal crossovers is 41.1.")

    print("*** Inheritance from grandparents ***")
    for c in range(4, 8):
      print('From grandparent ' + groupdict[c] + " you inherited " + "{:.1f}".format(
          100*ancestor_content[str(c)]/totbp+unallocgp/4) + "% of your genome.")
    print('Typical range of inheritance from grandparents is 14% - 35%.')

    print("*** Inheritance from great-grandparents ***")
    for c in range(8, 16):
      print('From great-grandparent ' + groupdict[c] + " you inherited " + "{:.1f}".format(
          100*ancestor_content[str(c)]/totbp+unallocggp/8) + "% of your genome.")
    print('Typical range of inheritance from great-grandparents is 4% - 22%.')


diagnostic_messages(ancestor_content, ancestor_segments, groupdict)

Invalid apikey, limiting match data to first 1,000.
*** Coverage ***
Coverage at grandparent generation is 77.1%
Coverage at great-grandparent generation is 71.6%
Coverage at 2nd great-grandparent generation is 49.2%
Coverage at 3rd great-grandparent generation is 10.9%


In [ ]:
#@title Export Results { display-mode: "form" }
#@markdown Export a CSV file with the results
from pandas.io.formats.style_render import DataFrame
from google.colab import files
try:
  df=DataFrame
  chr=list()
  start=list()
  end=list()
  group=list()
  color=list()
  ancestryside=list()
  colors = distinctipy.get_colors(64,rng=1)
  colorhex = {}
  for (ind, g) in enumerate(range(64)):
      colorhex[g] = get_hex(colors[ind])
  for g in  ancestor_segments:
    if int(g)>3:
      for c in ancestor_segments[g]:
        for s in ancestor_segments[g][c]:
          chr.append(c)
          start.append(s[0])
          end.append(s[1])
          group.append(groupdict[int(g)])
          color.append(colorhex[int(g)])
          ancestryside.append(side(int(g)))
  ddict = {'chr': chr, 'start': start, 'end': end, 'group':group,'color':color,'ancestry side':ancestryside} 
  df=DataFrame(ddict)
  df.to_csv('export.csv',index=False) 
  files.download('export.csv')
except Exception as e:
  print(str(e))
  print("Please run the 'Calculate your Ancestor Composition' cell first.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import requests
#@title Register for or recover an API key { display-mode: "form" }
#@markdown An API key allows you to run the solver using up to 30,000 matches, up to five times per day.  Without an API key, the solver uses only 1,000 matches.
EmailAddress = "" #@param {type:"string"}

def request_api_key(email):
  # Call the Ancestor Composition Solver API
  json_input=dict()
  json_input["Email"]=email
  url="https://5if5u6ou58.execute-api.us-east-1.amazonaws.com/default/AncestorComposition"
  r = requests.post(url, json=json_input)
  json_output=r.json()
  print(str(json_output["Message"]))

request_api_key(EmailAddress)

In [ ]:
#@markdown <a name="directions"></a>
#@title Detailed Directions
#@markdown Watch a quick demonstration [video](https://screencast-o-matic.com/watch/c36DcZVtdgF).  And here are detailed directions:
#@markdown 1.   Download matching segment data CSV files from [23andMe](https://www.23andme.com/), [MyHeritage](https://www.myheritage.com/), and/or [FamilyTree DNA](https://www.familytreedna.com/).
#@markdown 2.   For some of your matches (at least 25 - 300 individuals, for best results), identify the most recent common ancestor or ancestral couple that you share with the match.  This can be done using MyHeritage "Theory of Relativity", Ancestry "ThruLines", FamilyTree DNA, or 23AndMe "Family Tree".  Record this information in a "Common Ancestor" CSV file (download a template by running this cell) with five columns and a header row:
#@markdown    *   "Match Name", a column for which each row contains the names of a matching individual as it appears in a matching segment CSV file.
#@markdown    *   "Male Ancestor Name".  Each row of this column contains the name of the most recent male ancestor that you share with the matching individual.
#@markdown    *  "Male Ancestor #", the ahntentafel number (see diagram below) of the common male ancestor.
#@markdown    * "Female Ancestor Name".  Each row of this column contains the name of the most recent female ancestor that you share with the matching individual.
#@markdown    * "Female Ancestor #", the ahnentafel number of the common female ancestor.
#@markdown 3. Click on the folder icon to the left to expand the "Files" pane.
#@markdown 4. Click the "Upload to session storage" button which will appear in the upper left to upload CSV files for match shared segments (from MyHeritage, FamilyTree DNA, and/or 23andMe) and the CSV file (from Step 2) with common ancestor information for some of your matches.
#@markdown 5. Run the Ancestor Composition Solver.  The results will be displayed in:
#@markdown    * An ideogram showing which regions of your chromosomes were inherited from specific ancestors.
#@markdown   *  A fan chart showing the percentage of your genome inherited from ancestors in each generation.
#@markdown 6. You may export the results to a CSV file.
 

from pandas.io.formats.style_render import DataFrame
from google.colab import files
import pandas as pd
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.models import LabelSet, ColumnDataSource
import math
from bokeh.io import output_notebook
output_notebook()

df=DataFrame
ddict = {'Match Name': ["NameofMatch"], 'Male Ancestor Name': ["Father of Paternal Grandfater"], 'Male Ancestor #': [8], 'Female Ancestor Name':["Mother of Paternal Grandfather"],'Female Ancestor #':[9]} 
df=DataFrame(ddict)
df.to_csv('template.csv',index=False) 
files.download('template.csv')

def gen(anc):
    '''Generation of ancestor '''
    return math.floor(math.log(anc, 2))
#%%
gendata=list()
for ahn in range(1,128):
    if ahn == 1:
        adder=" (You)"
    elif ahn == 2:
        adder=" (Father)"
    elif ahn == 3:
        adder=" (Mother)"
    else:
        adder = ""
    thisgen=dict()
    thisgen["gen"]=gen(ahn)
    thisgen["label"]=str(ahn)+adder
    thisgen["innerrad"]=gen(ahn)
    thisgen["outerrad"]=gen(ahn)+1
    increment=math.pi/(2**gen(ahn))
    which=ahn-2**gen(ahn)
    thisgen["startangle"]=math.pi-(which+1)*increment 
    thisgen["endangle"]=math.pi-which*increment
    thisgen["midangle"]=(which+0.5)*increment
    thisgen['xlabel']=0.5*(thisgen["innerrad"]+thisgen["outerrad"])*math.cos(0.5*(thisgen["startangle"]+thisgen["endangle"]))
    thisgen['ylabel']=0.5*(thisgen["innerrad"]+thisgen["outerrad"])*math.sin(0.5*(thisgen["startangle"]+thisgen["endangle"]))
   
    gendata.append(thisgen)
data = pd.DataFrame(gendata)



p = figure(plot_height=1000, plot_width=1000, title="Ahnenetafel Numbers", toolbar_location=None,
           tools="hover", tooltips="@label", x_range=(-8, 8),y_range=(-8,8))
source = ColumnDataSource(data)
p.annular_wedge(x=0, y=0, inner_radius="innerrad", outer_radius="outerrad",
        start_angle="startangle", end_angle="endangle",
        line_color="black", fill_color="white",  source=source)


labels = LabelSet(x='xlabel', y='ylabel', text='label', source=source,
                  text_align ='center',text_baseline='center',text_font_size="9pt")

p.add_layout(labels)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

show(p)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### FAQ
1.   How does this notebook preserve my privacy?
    *   The CSV files that you upload are stored in a temporary private Google Colab workspace and are not available to others.
    *   The data is anonymized (all names removed) before a solver API is called to estimate your ancestral composition.
    *   This Colab notebook with your ancestral composition will be private to you unless you share it with others.
2.   How does the solver work?
    *   The solver assigns DNA segments to ancestors in a way that is:
        * **Self-consistent**.  Segments from one ancestor may overlap with segments from another ancestor only if the two ancestors are on the same line.
        * **Maximally consistent with common ancestor matches**.  The solver assigns segments to ancestors in a way that it is 100% consistent with the maximum possible number of common ancestor matches. It is generally not possible to be consistent with all common ancestor matches, due to phasing errors, false positives, and DNA shared from multiple lines.
        * **Supported by a large percentage of other match data**.  Ancestor segments from a match with an unknown common ancestor are used to expand the set of ancestor segments in a way that is self-consistent and fully consistent with known common ancestor matches.

3. How has the solver been validated?
    * Limited validation testing has shown that the results of the solver:
        * Are self-consistent.
        * Are maximally-consistent with common ancestor matches.
        * Are supported by a large percentage of other match data.
        * Predict a number of crossovers on the maternal and paternal lines that are within typical ranges, if sufficient common ancestor data is provided.
        * Predict inheritance percentages for grandparents and great-grandparents that are within typical ranges, if sufficient common ancestor data is provided.
    * Please note that the results can only be as accurate as the provided common ancestor information.

4. Who is the developer?
    * The developer is a professional data scientist and family history enthusiast.
    
5. Questions or problems running this notebook?
     * Please report problems at https://github.com/AncestorComposition/public/issues





### License

This notebook is licensed for personal, noncommercial use under the folllowing terms:
https://creativecommons.org/licenses/by-nc-nd/4.0/

The API called by this notebook may be used only for personal, noncommercial use.

Inquiries for commercial licenses may be sent to info@ancestorcomposition.com.